In [1]:
import os
import numpy as np 
import shutil 
import pandas as pd 

from PIL import Image
import matplotlib.pyplot as plt

In [20]:
root_path = '/home/wtan/Work/Datasets/Object-Detection-LVT/2024-Q1/data/merged_real_data_processed' 
out_root_path = root_path+'/cropped-person-images' 

os.makedirs(out_root_path, exist_ok = True) 

splits = ['train','valid','test']

os.makedirs(out_root_path+'/train/images', exist_ok = True) 
os.makedirs(out_root_path+'/test/images', exist_ok = True) 
os.makedirs(out_root_path+'/valid/images', exist_ok = True) 


In [14]:
nc: 9
names: ['animal', 'car', 'container', 'deer', 'dog', 'lvt', 'motorcycle', 'person', 'truck']
person_id = 6 

In [4]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = abs(max((xB - xA, 0)) * max((yB - yA), 0))
    if interArea == 0:
        return 0
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = abs((boxA[2] - boxA[0]) * (boxA[3] - boxA[1]))
    boxBArea = abs((boxB[2] - boxB[0]) * (boxB[3] - boxB[1]))

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

def bb_A_inside_B(boxA, boxB, margin=10):
    return boxA[0]>=boxB[0]-margin and boxA[1]>=boxB[1]-margin and boxA[2]<=boxB[2]+margin and boxA[3]<=boxB[3]+margin 


In [21]:
debug = False 

for split in ['test']:
    
    imgfiles = os.listdir(os.path.join(root_path, split, 'images')) 
    imgfiles = [x for x in imgfiles if x.split('.')[-1].lower() in ['jpg','jpeg','png']] 
    imgfiles.sort() 
    #print(len(imgfiles),imgfiles[:5]) 

    for f in imgfiles:

        #if not f == 'gettyimages-1202383097-612x612_jpg.rf.d8dd9b001767d8a2ae26a7a435746c9b.jpg':
        #    continue 

        
        fname = os.path.splitext(f)[0]  
        img_file = os.path.join(os.path.join(root_path, split, 'images',f))          
        label_file = os.path.join(os.path.join(root_path, split, 'labels',fname+'.txt'))
        print(img_file,label_file) 

        fp = open(label_file,'r')
        labels = fp.readlines() 
        fp.close() 
        labels = [x.strip() for x in labels] 

        #print("labels = ", labels) 

        im = Image.open(img_file)
        width, height = im.size

        if debug: 
            plt.imshow(im) 
            plt.show() 

        person_bboxs = [] 
        helmet_bboxs = [] 
        vest_bboxs = [] 

        for l in labels: 
            l = l.split(' ') 
            l = [float(x) for x in l] 

            if l[0]==0 or l[0]==1: #class 
                continue 

            w = int(l[3]*width) 
            h = int(l[4]*height) 
            x = int((l[1]-l[3]/2)*width)             
            y = int((l[2]-l[4]/2)*height) 

            if debug: 
                print(l,w,h,x,y)                 
                
            #im_person = im[y:y+h,x:x+w] #opencv or numpy 
            im_crop = im.crop((x,y,x+w,y+h))

            if l[0] == person_id:                                
                person_bboxs.append((x,y,x+w,y+h))
            '''    
            elif l[0]== helmet_id:
                helmet_bboxs.append((x,y,x+w,y+h))
            elif l[0]== vest_id:
                vest_bboxs.append((x,y,x+w,y+h))
            '''

        #match person, helmet, vest bboxes  
        bboxs_by_person = {}
        for i,pbox in enumerate(person_bboxs):
            bboxs_by_person[i] = {} 
            bboxs_by_person[i]['person'] = pbox            
            bboxs_by_person[i]['helmet'] = [] 
            bboxs_by_person[i]['vest'] = [] 

            for hbox in helmet_bboxs:
                bb_A_inside_B(hbox, pbox)
                bboxs_by_person[i]['helmet'].append(hbox)                
                continue 
            
            for vbox in vest_bboxs:
                bb_A_inside_B(vbox, pbox)
                bboxs_by_person[i]['vest'].append(vbox)                
                continue 
                

        #print(bboxs_by_person) 
        for i in bboxs_by_person: 
            b = bboxs_by_person[i] 
            #print(b) 
            pbox = b['person']
            im_person = im.crop(pbox)

            if debug: 
                plt.imshow(im_person) 
                plt.title('person')
                plt.show() 
                print("person bbox = ", pbox) 

            #generate bbox of helmet and vest in the person image  
            width, height = im_person.size
            #print('person image size = ', width, height) 

            '''
            labels = [] 
            for hbox in b['helmet']:
                
                if not bb_A_inside_B(hbox,pbox):
                    #print('hbox inside pbox = False') 
                    continue 
                left  = hbox[0]-pbox[0]
                upper = hbox[1]-pbox[1]
                right = hbox[2]-pbox[0] 
                lower = hbox[3]-pbox[1] 
                x = (left+right)/2/width 
                y = (upper+lower)/2/height 
                w = (right-left)/width 
                h = (lower-upper)/height 

                if debug: 
                    print("hbox = ", hbox) 
                    print((x,y,w,h))
                    im_helmet = im.crop(hbox)
                    plt.imshow(im_helmet) 
                    plt.title('helmet') 
                    plt.show() 

                #there is still possible mismatched helmet and person 
                if x<0 or x>0.99 or y<0 or y>0.99 or w<0 or w>0.99 or h<0 or h>0.99:
                    continue 
                
                labels.append((helmet_id_new,x,y,w,h)) 
                

            #print("debug vbox, pbox = ", pbox)
            for vbox in b['vest']:
                if not bb_A_inside_B(vbox,pbox):
                    #print(vbox) 
                    #print('vbox inside pbox = False') 
                    continue 
                    
                left  = hbox[0]-pbox[0]
                upper = hbox[1]-pbox[1]
                right = hbox[2]-pbox[0] 
                lower = hbox[3]-pbox[1] 
                x = (left+right)/2/width 
                y = (upper+lower)/2/height 
                w = (right-left)/width 
                h = (lower-upper)/height 
                if debug: 
                    #print("vbox = ", vbox) 
                    #print((x,y,w,h))
                    im_vest = im.crop(vbox)
                    plt.imshow(im_vest) 
                    plt.title('vest') 
                    plt.show() 

                #there is still possible mismatched helmet and person 
                if x<0 or x>0.99 or y<0 or y>0.99 or w<0 or w>0.99 or h<0 or h>0.99:
                    continue 
                    
                labels.append((vest_id_new,x,y,w,h)) 
            '''

            #write images and label files 
            out_imgfile = fname + '_{:03d}.jpg'.format(i) 
            out_labelfile = fname + '_{:03d}.txt'.format(i)

            out_imgfile = os.path.join(out_root_path,split,'images',out_imgfile)
            #out_labelfile = os.path.join(out_root_path,split,'labels',out_labelfile)

            #print(out_imgfile) 
            #print(out_labelfile) 
            
            im_person.save(out_imgfile)
            
            '''
            fp = open(out_labelfile,'w') 
            for l in labels:
                fp.write('{} {} {} {} {}\n'.format(l[0],l[1],l[2],l[3],l[4])) 
            fp.close() 
            '''

/home/wtan/Work/Datasets/Object-Detection-LVT/2024-Q1/data/merged_real_data_processed/test/images/15211_0_jpg.rf.80dc7698918dd2b311eb338f3478617f.jpg /home/wtan/Work/Datasets/Object-Detection-LVT/2024-Q1/data/merged_real_data_processed/test/labels/15211_0_jpg.rf.80dc7698918dd2b311eb338f3478617f.txt
/home/wtan/Work/Datasets/Object-Detection-LVT/2024-Q1/data/merged_real_data_processed/test/images/15764_0_jpg.rf.4c27ef4636359f7367cfba3053ef8c11.jpg /home/wtan/Work/Datasets/Object-Detection-LVT/2024-Q1/data/merged_real_data_processed/test/labels/15764_0_jpg.rf.4c27ef4636359f7367cfba3053ef8c11.txt
/home/wtan/Work/Datasets/Object-Detection-LVT/2024-Q1/data/merged_real_data_processed/test/images/17741_0_jpg.rf.4a9f61cbd6cd48a4d71336fc7aec7398.jpg /home/wtan/Work/Datasets/Object-Detection-LVT/2024-Q1/data/merged_real_data_processed/test/labels/17741_0_jpg.rf.4a9f61cbd6cd48a4d71336fc7aec7398.txt
/home/wtan/Work/Datasets/Object-Detection-LVT/2024-Q1/data/merged_real_data_processed/test/images/18